In [1]:
!pip install rdkit
!pip install pyg-lib torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.13.0+cu117.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 27.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.0+cu117.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 877.7/877.7 KB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.0/565.0 KB 19.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 KB 24.5 MB/s eta 0:00:00
  Created wheel for torch-geometric: filename=torch_

In [2]:
import numpy as np
import pandas as pd
import rdkit.Chem as Chem
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics as metrics
import torch 
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from torch_geometric.data import Data
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
from torch_geometric.loader import DataLoader
import torch_geometric
from torch_geometric.nn import GCNConv
import os
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

In [3]:
!mkdir data
!mkdir /content/data/raw
!mkdir /content/data/processed

In [4]:
!cp "/content/drive/MyDrive/Colab Notebooks/Job_test/cmpd.csv" "/content/data/raw"

In [6]:
cmpd_df = pd.read_csv('/content/data/raw/cmpd.csv')
cmpd_df.head()

,inchikey,smiles,group,activity
0,FNHKPVJBJVTLMP-UHFFFAOYSA-N,CNC(=O)c1cc(Oc2ccc(NC(=O)Nc3ccc(Cl)c(C(F)(F)F)...,train,active
1,CUDVHEFYRIWYQD-UHFFFAOYSA-N,CNC(=O)c1cccc2cc(Oc3ccnc4cc(OCC5(N)CC5)c(OC)cc...,train,active
2,TTZSNFLLYPYKIL-UHFFFAOYSA-N,Cc1cc2cc(Oc3ccnc(Nc4cccc(CS(=O)(=O)NCCN(C)C)c4...,test,active
3,UOVCGJXDGOGOCZ-UHFFFAOYSA-N,COc1cc2c(cc1F)C(c1ccccc1Cl)=Nc1c(n[nH]c1C)N2,train,active
4,CUIHSIWYWATEQL-UHFFFAOYSA-N,Cc1ccc(Nc2nccc(N(C)c3ccc4c(C)n(C)nc4c3)n2)cc1S...,test,active


In [7]:
train_df = cmpd_df[cmpd_df['group'] == "train"] 
test_df = cmpd_df[cmpd_df['group'] == "test"] 
train_df.reset_index(inplace = True,drop = True)
test_df.reset_index(inplace = True,drop = True)

le=LabelEncoder()
train_df["activity"]=le.fit_transform(train_df["activity"])
test_df["activity"]=le.fit_transform(test_df["activity"])

train_df["activity"] = train_df["activity"].astype(int)
test_df["activity"] = test_df["activity"].astype(int)

train_df.to_csv("/content/data/raw/train.csv")
test_df.to_csv("/content/data/raw/test.csv")

<ipython-input-7-ac65f67fca99>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["activity"]=le.fit_transform(train_df["activity"])
<ipython-input-7-ac65f67fca99>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["activity"]=le.fit_transform(test_df["activity"])
<ipython-input-7-ac65f67fca99>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

In [8]:
# label = np.asarray([train_df["activity"]])


In [9]:
# Count each class data points
train_df["activity"].value_counts()

0    2146
1    1075
3     486
2     270
Name: activity, dtype: int64

In [10]:
import torch
from torch_geometric.data import Dataset, download_url


class MoleculeDataset(Dataset):
    def __init__(self, root, filename, transform=None, test=False, pre_transform=None, pre_filter=None):
        """
        root = Where the dataset should be stored. This folder is split
        into raw_dir (downloaded dataset) and processed_dir (processed data). 
        """
        self.test = test
        self.filename = filename
        super(MoleculeDataset, self).__init__(root, transform, pre_transform, pre_filter)

    @property
    def raw_file_names(self):
        """ If this file exists in raw_dir, the download is not triggered.
            (The download func. is not implemented here)  
        """
        return self.filename

    @property
    def processed_file_names(self):
        """ If these files are found in raw_dir, processing is skipped"""
        self.data = pd.read_csv(self.raw_paths[0]).reset_index()

        if self.test:
            return [f'data_test_{i}.pt' for i in list(self.data.index)]
        else:
            return [f'data_{i}.pt' for i in list(self.data.index)]

    def download(self):
        # Download to `self.raw_dir`.
        pass

    def process(self):
        self.data = pd.read_csv(self.raw_paths[0])
        for index, mol in tqdm(self.data.iterrows(), total=self.data.shape[0]):
            mol_obj = Chem.MolFromSmiles(mol["smiles"])
            # Get node features
            node_feats = self._get_node_features(mol_obj)
            # Get edge features
            edge_feats = self._get_edge_features(mol_obj)
            # Get adjacency info
            edge_index = self._get_adjacency_info(mol_obj)
            # Get labels info
            label = self._get_labels(mol["activity"])

            # Create data object
            data = Data(x=node_feats, 
                        edge_index=edge_index,
                        edge_attr=edge_feats,
                        y=label,
                        smiles=mol["smiles"]
                        ) 
            if self.test:
                torch.save(data, 
                    os.path.join(self.processed_dir, 
                                 f'data_test_{index}.pt'))
            else:
                torch.save(data, 
                    os.path.join(self.processed_dir, 
                                 f'data_{index}.pt'))

    def _get_node_features(self, mol):
        """ 
        This will return a matrix / 2d array of the shape
        [Number of Nodes, Node Feature size]
        """
        all_node_feats = []

        for atom in mol.GetAtoms():
            node_feats = []
            # Feature 1: Atomic number        
            node_feats.append(atom.GetAtomicNum())
            # Feature 2: Atom degree
            node_feats.append(atom.GetDegree())
            # Feature 3: Formal charge
            node_feats.append(atom.GetFormalCharge())
            # Feature 4: Hybridization
            node_feats.append(atom.GetHybridization())
            # Feature 5: Aromaticity
            node_feats.append(atom.GetIsAromatic())
            # Feature 6: Total Num Hs
            node_feats.append(atom.GetTotalNumHs())
            # Feature 7: Radical Electrons
            node_feats.append(atom.GetNumRadicalElectrons())
            # Feature 8: In Ring
            node_feats.append(atom.IsInRing())
            # Feature 9: Chirality
            node_feats.append(atom.GetChiralTag())

            # Append node features to matrix
            all_node_feats.append(node_feats)

        all_node_feats = np.asarray(all_node_feats)
        return torch.tensor(all_node_feats, dtype=torch.float)

    def _get_edge_features(self, mol):
        """ 
        This will return a matrix / 2d array of the shape
        [Number of edges, Edge Feature size]
        """
        all_edge_feats = []

        for bond in mol.GetBonds():
            edge_feats = []
            # Feature 1: Bond type (as double)
            edge_feats.append(bond.GetBondTypeAsDouble())
            # Feature 2: Rings
            edge_feats.append(bond.IsInRing())
            # Append node features to matrix (twice, per direction)
            all_edge_feats += [edge_feats, edge_feats]

        all_edge_feats = np.asarray(all_edge_feats)
        return torch.tensor(all_edge_feats, dtype=torch.float)

    def _get_adjacency_info(self, mol):
        """
        We could also use rdmolops.GetAdjacencyMatrix(mol)
        but we want to be sure that the order of the indices
        matches the order of the edge features
        """
        edge_indices = []
        for bond in mol.GetBonds():
            i = bond.GetBeginAtomIdx()
            j = bond.GetEndAtomIdx()
            edge_indices += [[i, j], [j, i]]

        edge_indices = torch.tensor(edge_indices)
        edge_indices = edge_indices.t().to(torch.long).view(2, -1)
        return edge_indices

    def _get_labels(self, label):
        label = np.asarray([label])
        label = label.astype(np.int64)
        return torch.tensor(label, dtype=torch.int64)

    def len(self):
        return self.data.shape[0]

    def get(self, idx):
        """ - Equivalent to __getitem__ in pytorch
            - Is not needed for PyG's InMemoryDataset
        """
        if self.test:
            data = torch.load(os.path.join(self.processed_dir, 
                                 f'data_test_{idx}.pt'))
        else:
            data = torch.load(os.path.join(self.processed_dir, 
                                 f'data_{idx}.pt'))        
        return data

In [12]:
# dataset = MoleculeDataset(root = "/content/data/")

In [15]:
# print(dataset[0].edge_index.t())
# print(dataset[0].x)
# print(dataset[0].edge_attr)
# print(dataset[0].y)


# **GNN Architecture**

In [20]:
import torch
import torch.nn.functional as F 
from torch.nn import Linear, BatchNorm1d, ModuleList
from torch_geometric.nn import TransformerConv, TopKPooling 
from torch_geometric.nn import GATConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from sklearn.metrics import confusion_matrix, f1_score, \
    accuracy_score, precision_score, recall_score, roc_auc_score
import numpy as np
from tqdm import tqdm
torch.manual_seed(42)

In [37]:
class GNN(torch.nn.Module):
    def __init__(self, feature_size):
        super(GNN, self).__init__()
        num_classes = 4
        embedding_size = 1024

        # GNN layer
        self.conv1 = GATConv(feature_size, embedding_size, heads=3, dropout=0.3)
        self.head_transform1 = Linear(embedding_size * 3, embedding_size)
        self.pool1 = TopKPooling(embedding_size, ratio = 0.8)
        self.conv2 = GATConv(embedding_size, embedding_size, heads=3, dropout=0.3)
        self.head_transform2 = Linear(embedding_size*3, embedding_size)
        self.pool2 = TopKPooling(embedding_size, ratio=0.5)
        self.conv3 = GATConv(embedding_size, embedding_size, heads=3, dropout=0.3)
        self.head_transform3 = Linear(embedding_size*3, embedding_size)
        self.pool3 = TopKPooling(embedding_size, ratio=0.2)

        # Linear layers
        self.linear1 = Linear(embedding_size*2, 1024)
        self.linear2 = Linear(1024, num_classes)

    def forward(self, x, edge_attr, edge_index, batch_index):
        # First block
        x = self.conv1(x, edge_index)
        x = self.head_transform1(x)

        x, edge_index, edge_attr, batch_index, _, _ = self.pool1(x, edge_index,
                                                                 None, batch_index)
        x1 = torch.cat([gmp(x, batch_index), gap(x, batch_index)], dim=1)

        #second block
        x = self.conv2(x, edge_index)
        x = self.head_transform2(x)
        x, edge_index, edge_attr, batch_index, _, _ = self.pool2(x, edge_index, None, batch_index)
        x2 = torch.cat([gmp(x, batch_index), gap(x, batch_index)], dim=1)

        # third block
        x = self.conv3(x, edge_index)
        x = self.head_transform3(x)
        x, edge_index, edge_attr, batch_index, _, _ = self.pool3(x, edge_index, None, batch_index)
        x3 = torch.cat([gmp(x, batch_index), gap(x, batch_index)], dim=1)

        # Concat pooled vectors
        x = x1 + x2 + x3

        # output block
        x = self.linear1(x).relu()
        x = F.dropout(x, p=0.5, training = self.training)
        x = self.linear2(x)

        return x


# **Training the data**

In [38]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [18]:
train_dataset = MoleculeDataset(root="/content/data/", filename="train.csv")
test_dataset = MoleculeDataset(root="/content/data/", filename="test.csv")

Processing...
100%|██████████| 3977/3977 [00:14<00:00, 279.48it/s]
Done!


In [39]:
def count_parameters(model):  
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [48]:
# Loading the model

model = GNN(feature_size=train_dataset[0].x.shape[1])
model = model.to(device)
print(f"Number of parameters: {count_parameters(model)}")
model

Number of parameters: 17892356


GNN(
  (conv1): GATConv(9, 1024, heads=3)
  (head_transform1): Linear(in_features=3072, out_features=1024, bias=True)
  (pool1): TopKPooling(1024, ratio=0.8, multiplier=1.0)
  (conv2): GATConv(1024, 1024, heads=3)
  (head_transform2): Linear(in_features=3072, out_features=1024, bias=True)
  (pool2): TopKPooling(1024, ratio=0.5, multiplier=1.0)
  (conv3): GATConv(1024, 1024, heads=3)
  (head_transform3): Linear(in_features=3072, out_features=1024, bias=True)
  (pool3): TopKPooling(1024, ratio=0.2, multiplier=1.0)
  (linear1): Linear(in_features=2048, out_features=1024, bias=True)
  (linear2): Linear(in_features=1024, out_features=4, bias=True)
)

In [49]:
# Loss and Optimizer 
# weights = torch.tensor([1,10], dtype=torch.float32).to(device)
# loss_fn = torch.nn.CrossEntropyLoss(weight=weights)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)

In [50]:
# NUM_GRAPHS_PER_BATCH = 256
NUM_GRAPHS_PER_BATCH = 64
train_loader = DataLoader(train_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle = True)

In [57]:
def train(epoch):
  # Enumerate over the data
    all_preds = []
    all_labels = []
    running_loss = 0.0
    # step = 0
    for _, batch in enumerate(tqdm(train_loader)):
        # Use GPU
        batch.to(device)  
        # Reset gradients
        optimizer.zero_grad() 
        # Passing the node features and the connection info
        pred = model(batch.x.float(), 
                                batch.edge_attr.float(),
                                batch.edge_index, 
                                batch.batch) 
        # Calculating the loss and gradients
        loss = torch.sqrt(loss_fn(pred, batch.y))
        loss.backward()  
        optimizer.step()  
        # Update tracking
        running_loss += loss.item()
        # step += 1
        all_preds.append(np.argmax(pred.cpu().detach().numpy(), axis=1))
        all_labels.append(batch.y.cpu().detach().numpy())
    all_preds = np.concatenate(all_preds).ravel()
    all_labels = np.concatenate(all_labels).ravel()
    calculate_metrics(all_preds, all_labels, epoch, "train")
    return loss

In [55]:
def test(epoch):
    all_preds = []
    all_labels = []
    running_loss = 0.0
    step = 0
    for batch in test_loader:
        batch.to(device)  
        pred = model(batch.x.float(), 
                        batch.edge_attr.float(),
                        batch.edge_index, 
                        batch.batch) 
        loss = torch.sqrt(loss_fn(pred, batch.y))
        all_preds.append(np.argmax(pred.cpu().detach().numpy(), axis = 1))
        all_labels.append(batch.y.cpu().detach().numpy())
    
    all_preds = np.concatenate(all_preds).ravel()
    all_labels = np.concatenate(all_labels).ravel()
    calculate_metrics(all_preds, all_labels, epoch, "test")
    return loss

In [56]:
def calculate_metrics(y_pred, y_true, epoch, type):
    print(f"\n Confusion matrix: \n {confusion_matrix(y_pred, y_true)}")
    print(f"F1 Score: {f1_score(y_true, y_pred, average='macro')}")
    print(f"Accuracy: {accuracy_score(y_true, y_pred)}")
    prec = precision_score(y_true, y_pred, average='macro')
    rec = recall_score(y_true, y_pred, average='macro')
    print(f"Precision: {prec}")
    print(f"Recall: {rec}")
    # mlflow.log_metric(key=f"Precision-{type}", value=float(prec), step=epoch)
    # mlflow.log_metric(key=f"Recall-{type}", value=float(rec), step=epoch)
    try:
        roc = roc_auc_score(y_true, y_pred)
        print(f"ROC AUC: {roc}")
        # mlflow.log_metric(key=f"ROC-AUC-{type}", value=float(roc), step=epoch)
    except:
        # mlflow.log_metric(key=f"ROC-AUC-{type}", value=float(0), step=epoch)
        print(f"ROC AUC: notdefined")

In [ ]:
for epoch in range(300): 
    # Training
    model.train()
    loss = train(epoch=epoch)
    loss = loss.detach().cpu().numpy()
    print(f"Epoch {epoch} | Train Loss {loss}")

    # Testing
    model.eval()
    if epoch % 5 == 0:
        loss = test(epoch = epoch)
        loss = loss.detach().cpu().numpy()
        print(f"Epochs {epoch} | test Loss {loss}")

    scheduler.step()
print("Done!")
    # mlflow.log_metric(key="Train loss", value=float(loss), step=epoch)

    # Testing
    # model.eval()
    # if epoch % 5 == 0:
    #         loss = test(epoch, model, test_loader, loss_fn)
    #         print(f"Epoch {epoch} | Test Loss {loss}")
    #         mlflow.log_metric(key="Test loss", value=float(loss), step=epoch)
            
    #         # Update best loss
    #         if float(loss) < best_loss:
    #             best_loss = loss
    #             # Save the currently best model 
    #             mlflow.pytorch.log_model(model, "model", signature=SIGNATURE)
    #             early_stopping_counter = 0
    #         else:
    #             early_stopping_counter += 1

    #     scheduler.step()
    # else:
    #     print("Early stopping due to no improvement.")
    #     return [best_loss]
    # print(f"Finishing training with best test loss: {best_loss}")
    # return [best_loss]

100%|██████████| 63/63 [03:45<00:00,  3.58s/it]
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



 Confusion matrix: 
 [[2142 1068  269  484]
 [   4    7    1    2]
 [   0    0    0    0]
 [   0    0    0    0]]
F1 Score: 0.17852906661519885
Accuracy: 0.5403570530550666
Precision: 0.2601249053747161
Recall: 0.25116192375214025
ROC AUC: notdefined
Epoch 0 | Train Loss 1.0346851348876953


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



 Confusion matrix: 
 [[578 658 109 208]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]]
F1 Score: 0.1356170811825434
Accuracy: 0.37218287186091437
Precision: 0.09304571796522859
Recall: 0.25
ROC AUC: notdefined
Epochs 0 | test Loss 1.1075352430343628


100%|██████████| 63/63 [03:41<00:00,  3.52s/it]
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

100%|██████████| 63/63 [03:45<00:00,  3.58s/it]



 Confusion matrix: 
 [[1726  513  159  260]
 [ 398  548  101  176]
 [   3    0    0    1]
 [  19   14   10   49]]
F1 Score: 0.3412636249048881
Accuracy: 0.5841086245914006
Precision: 0.4075119031311216
Recall: 0.3537193831985777
ROC AUC: notdefined
Epoch 26 | Train Loss 0.9087729454040527


100%|██████████| 63/63 [03:41<00:00,  3.52s/it]



 Confusion matrix: 
 [[1759  564  165  299]
 [ 365  504  101  148]
 [   0    0    0    1]
 [  22    7    4   38]]
F1 Score: 0.327311464889867
Accuracy: 0.578576816696002
Precision: 0.40429021911960394
Recall: 0.34167275044803336
ROC AUC: notdefined
Epoch 27 | Train Loss 1.069650650024414


 54%|█████▍    | 34/63 [02:03<01:38,  3.39s/it]